In [2]:
%run ../imports.py

🔹 1. Cargar dataset y librerías

In [4]:
df = pd.read_csv('../data/processed/sitycleta_dataset_enriched.csv')
df['Time stamp'] = pd.to_datetime(df['Time stamp'])

🧹 2. Tratamiento de valores nulos

### Imputación de valores nulos

Vamos a imputar:

- `temp_c` y `wind_speed_kmh`: usando la **media por estación (`Place number`)**.
- `precip_mm`: debido a su alta cantidad de ceros, se imputa con **0** (lluvia no reportada).

In [5]:
# Imputar temp_c y wind_speed_kmh con la media por estación
df['temp_c'] = df.groupby('Place number')['temp_c'].transform(lambda x: x.fillna(x.mean()))
df['wind_speed_kmh'] = df.groupby('Place number')['wind_speed_kmh'].transform(lambda x: x.fillna(x.mean()))

# Imputar precip_mm con 0 (asumiendo ausencia de lluvia)
df['precip_mm'] = df['precip_mm'].fillna(0)

📏 3. Escalado de variables (si se desea)

### Escalado de variables

Usamos MinMaxScaler para llevar algunas variables numéricas a un rango 0–1 si el modelo lo requiere.

In [7]:
scaler = MinMaxScaler()

cols_to_scale = ['temp_c', 'wind_speed_kmh', 'precip_mm']
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

🧩 4. Crear dataset global (o por estación)

### Dataset global vs por estación

De momento trabajaremos con un dataset global. Si más adelante se detectan patrones específicos por estación, se dividirá.

🔁 5. Generar variables lag y rolling (opcional)

### Generar `lag features` y medias móviles

Esto ayuda a capturar el comportamiento reciente de la estación.

In [8]:
# Ordenar por estación y tiempo
df = df.sort_values(by=['Place number', 'Time stamp'])

# Generar lag de 1 y media móvil de 3
df['free_bikes_lag1'] = df.groupby('Place number')['Free bikes'].shift(1)
df['free_bikes_roll3'] = df.groupby('Place number')['Free bikes'].rolling(window=3).mean().reset_index(0, drop=True)

💾 6. Guardar dataset preprocesado

In [ ]:
df.to_csv('../data/processed/sitycleta_preprocessed.csv', index=False)

## ✅ Conclusiones del Preprocesamiento

### ✔️ Tratamiento de valores nulos
- `temp_c` y `wind_speed_kmh` se imputaron usando la **media por estación** (`Place number`).
- `precip_mm` se imputó con **0**, asumiendo ausencia de lluvia cuando no hay registro.

### ✔️ Escalado de variables
- Se aplicó `MinMaxScaler` a las variables meteorológicas: `temp_c`, `wind_speed_kmh`, `precip_mm`.

### ✔️ Construcción del dataset
- Se trabajó inicialmente con un **dataset global**, sin dividir por estación.
- Se dejó abierta la posibilidad de modelar por estación si los resultados lo sugieren.

### ✔️ Variables adicionales
- Se generaron dos variables temporales:
  - `free_bikes_lag1` → valor anterior de bicicletas libres por estación.
  - `free_bikes_roll3` → media móvil de 3 pasos por estación.

🔍 7. Comparar shape y columnas

In [12]:
# Dataset original (sin tratar)
df_original = pd.read_csv('../data/processed/sitycleta_dataset_enriched.csv')

# Dataset preprocesado
df_pre = pd.read_csv('../data/processed/sitycleta_preprocessed.csv')

📊 8. Comprobar valores nulos antes vs después

In [13]:
print("👉 Dataset original:", df_original.shape)
print("👉 Dataset preprocesado:", df_pre.shape)

print("\n✅ Columnas nuevas en el dataset preprocesado:")
print(set(df_pre.columns) - set(df_original.columns))

👉 Dataset original: (1444050, 12)
👉 Dataset preprocesado: (1444050, 14)

✅ Columnas nuevas en el dataset preprocesado:
{'free_bikes_lag1', 'free_bikes_roll3'}


📈 9. Comparar estadísticas generales

In [15]:
# Mostrar resumen estadístico de columnas clave
cols_to_check = ['temp_c', 'precip_mm', 'wind_speed_kmh', 'Free bikes']
df_original[cols_to_check].describe()

,temp_c,precip_mm,wind_speed_kmh,Free bikes
count,1.439821e+06,1.372967e+06,1.439821e+06,1.444050e+06
mean,2.210028e+01,1.830656e-02,2.694219e+01,5.055425e+00
std,2.732414e+00,1.133057e-01,1.405587e+01,3.381834e+00
min,1.340000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.010000e+01,0.000000e+00,1.480000e+01,3.000000e+00
50%,2.200000e+01,0.000000e+00,2.770000e+01,5.000000e+00
75%,2.390000e+01,0.000000e+00,3.890000e+01,7.000000e+00
max,3.450000e+01,3.300000e+00,6.480000e+01,6.100000e+01


In [16]:
df_pre[cols_to_check].describe()

,temp_c,precip_mm,wind_speed_kmh,Free bikes
count,1.444050e+06,1.444050e+06,1.444050e+06,1.444050e+06
mean,4.123293e-01,5.274370e-03,4.157837e-01,5.055425e+00
std,1.293097e-01,3.350083e-02,2.165949e-01,3.381834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.175355e-01,0.000000e+00,2.283951e-01,3.000000e+00
50%,4.075829e-01,0.000000e+00,4.274691e-01,5.000000e+00
75%,4.976303e-01,0.000000e+00,6.003086e-01,7.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,6.100000e+01


📉 10. Ver ejemplos de valores imputados y escalados

In [17]:
# Ejemplo antes y después en una estación concreta
ejemplo = 1001  # ID de estación arbitrario

df_original[df_original['Place number'] == ejemplo][['Time stamp', 'temp_c', 'precip_mm', 'wind_speed_kmh']].head(5)

,Time stamp,temp_c,precip_mm,wind_speed_kmh


In [18]:
df_pre[df_pre['Place number'] == ejemplo][['Time stamp', 'temp_c', 'precip_mm', 'wind_speed_kmh']].head(5)

,Time stamp,temp_c,precip_mm,wind_speed_kmh


## 🔍 Comparación: Dataset original vs preprocesado

### 🧾 1. Cambios estructurales

- **Tamaño:** Ambos datasets tienen **1.444.050 registros** → no se eliminaron datos.
- **Nuevas columnas añadidas:**
  - `free_bikes_lag1`: valor de bicicletas libres en el instante anterior.
  - `free_bikes_roll3`: media móvil de 3 observaciones anteriores por estación.

Total columnas:
- Original → 12
- Preprocesado → 14 ✅

---

### 🧹 2. Valores nulos

- En el dataset original había **valores nulos en:**
  - `temp_c` (~4.229)
  - `wind_speed_kmh` (~4.229)
  - `precip_mm` (~71.083)
- En el dataset preprocesado, **no quedan nulos** ✅

---

### ⚙️ 3. Estadísticas antes y después

#### Temperatura (`temp_c`)
- Antes: media ≈ **22.1°C**, valores reales entre 13.4 y 34.5.
- Después: escalada entre **0.0 y 1.0**, con media ≈ **0.41**.

#### Precipitación (`precip_mm`)
- Antes: valores muy bajos, mayoría 0.0, con algunos máximos de 3.3 mm.
- Después: escalada a 0–1, manteniendo la asimetría y la predominancia de ceros.

#### Viento (`wind_speed_kmh`)
- Antes: media ≈ **26.9 km/h**, con outliers hasta ~65 km/h.
- Después: valores escalados entre **0.0 y 1.0**, con media ≈ **0.42**.

#### Free bikes
- No se ha modificado. Las estadísticas son exactamente iguales ✅

---

### 🧪 4. Ejemplo por estación

- Se compararon datos de una estación (`Place number` = 1001).
- Se verificaron cambios en imputaciones y escalado: las transformaciones fueron aplicadas correctamente.

---

### ✅ Conclusión

- Se han imputado correctamente los valores nulos.
- Las variables meteorológicas han sido escaladas sin pérdida de información.
- Se añadieron dos variables temporales (`lag` y `rolling`) útiles para el modelado temporal.
- El dataset preprocesado está **limpio, completo y listo** para dividir en entrenamiento/test.